In [1]:
%pip install numpy, pandas, matplotlib, sklearn

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'numpy,'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.switch_backend('TkAgg')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree

In [3]:
diabet_dataset = pd.read_csv('Dataset/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
diabet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [4]:
nan_count = int(diabet_dataset.shape[0] * 0.1)

# Вибір випадкових індексів для заповнення NaN
nan_indices = np.random.choice(diabet_dataset.index, nan_count, replace=False)

# Заповнення вибраних рядків усіма стовпцями NaN
diabet_dataset.loc[nan_indices, :] = np.nan

# Вивід результату
print("Dataset with missing values (10% NaN and 90% not null):")
print("Dataset with missing values:")
diabet_dataset.info()

Dataset with missing values (10% NaN and 90% not null):
Dataset with missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       63623 non-null  float64
 1   HighBP                63623 non-null  float64
 2   HighChol              63623 non-null  float64
 3   CholCheck             63623 non-null  float64
 4   BMI                   63623 non-null  float64
 5   Smoker                63623 non-null  float64
 6   Stroke                63623 non-null  float64
 7   HeartDiseaseorAttack  63623 non-null  float64
 8   PhysActivity          63623 non-null  float64
 9   Fruits                63623 non-null  float64
 10  Veggies               63623 non-null  float64
 11  HvyAlcoholConsump     63623 non-null  float64
 12  AnyHealthcare         63623 non-null  float64
 13  NoDocbcCost           63623 non-null

In [5]:

# Розбиття віку на інтервали до 20 років по 5 років
age_intervals = [(i, i + 5) for i in range(0, 20, 5)]
age_group_counts = []

# Підрахунок кількості людей для кожного інтервалу
for interval in age_intervals:
    count = diabet_dataset[(diabet_dataset['Age'] >= interval[0]) & (diabet_dataset['Age'] < interval[1])].shape[0]
    age_group_counts.append(count)

# Побудова графіка кількості людей по інтервалам до 20 років
plt.figure(figsize=(8, 5))
plt.bar([f"{i[0]}-{i[1]}" for i in age_intervals], age_group_counts, color='skyblue')
plt.xlabel('Age Intervals')
plt.ylabel('Number of People')
plt.title('Number of People by Age Intervals (0-20 years)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [6]:
sex_counts = diabet_dataset['Sex'].value_counts()

# Побудова стовпчатої діаграми
plt.figure(figsize=(6, 4))
sex_counts.plot(kind='bar', color=['lightcoral', 'skyblue'])
plt.xlabel('Sex (1 - Female, 0 - Male)')
plt.ylabel('Number of People')
plt.title('Distribution of People by Sex')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()
# Виведення кількості
print("Кількість Жінок (Sex = 0):", sex_counts[1])
print("Кількість Чоловіків (Sex = 1):", sex_counts[0])

Кількість Жінок (Sex = 0): 29047
Кількість Чоловіків (Sex = 1): 34576


In [7]:
diabet_dataset['Diabetes_binary'] = diabet_dataset['Diabetes_binary'].astype('category')

# Побудова стовпчастої діаграми для відображення кількості людей з діабетом
plt.figure(figsize=(8, 6))
diabet_dataset['Diabetes_binary'].value_counts().plot(kind='bar', color='skyblue')
plt.xlabel('Diabetes Status')
plt.ylabel('Count')
plt.title('Number of People with Diabetes')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Побудова кругової діаграми для відображення кількості людей з діабетом
plt.figure(figsize=(6, 6))
diabet_dataset['Diabetes_binary'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['lightblue', 'lightgreen', 'lightcoral'])
plt.title('Percentage of People with Diabetes')
plt.ylabel('')
plt.tight_layout()
plt.show()

In [8]:
numeric_columns = diabet_dataset.select_dtypes(include=np.number).columns.tolist()

# Функція для обрахунку медіани, середнього значення і моди
def calculate_statistics(column_name):
    mean_value = diabet_dataset[column_name].mean()
    median_value = diabet_dataset[column_name].median()
    mode_value = diabet_dataset[column_name].mode().values[0] if not diabet_dataset[column_name].mode().empty else np.nan
    return mean_value, median_value, mode_value

# Створення DataFrame для збереження результатів обрахунків
statistics_df = pd.DataFrame(columns=['mean', 'median', 'mode'])

# Обрахунок статистики для кожного числового стовпця
for column in numeric_columns:
    mean_val, median_val, mode_val = calculate_statistics(column)
    statistics_df.loc[column] = [mean_val, median_val, mode_val]

# Виведення результатів
print("Summary Statistics:")
print(statistics_df)

Summary Statistics:
                           mean  median  mode
HighBP                 0.563523     1.0   1.0
HighChol               0.525250     1.0   1.0
CholCheck              0.975213     1.0   1.0
BMI                   29.847712    29.0  27.0
Smoker                 0.474687     0.0   0.0
Stroke                 0.062022     0.0   0.0
HeartDiseaseorAttack   0.147368     0.0   0.0
PhysActivity           0.702136     1.0   1.0
Fruits                 0.611634     1.0   1.0
Veggies                0.788457     1.0   1.0
HvyAlcoholConsump      0.042390     0.0   0.0
AnyHealthcare          0.955063     1.0   1.0
NoDocbcCost            0.093740     0.0   0.0
GenHlth                2.834510     3.0   3.0
MentHlth               3.750389     0.0   0.0
PhysHlth               5.798249     0.0   0.0
DiffWalk               0.252283     0.0   0.0
Sex                    0.456549     0.0   0.0
Age                    8.583248     9.0  10.0
Education              4.921129     5.0   6.0
Income        

In [10]:
X_train, X_test, y_train, y_test = train_test_split(diabet_dataset[['Diabetes_binary']], diabet_dataset['Sex'], test_size=0.3, random_state=42)

# Підрахунок кількості пацієнтів з діабетом у тренувальному та тестовому наборах
train_diabetes_counts = X_train[X_train['Diabetes_binary'] == 1].shape[0]
test_diabetes_counts = X_test[X_test['Diabetes_binary'] == 1].shape[0]

# Побудова діаграми
fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(['Train Diabetes'], [train_diabetes_counts], color='skyblue', label='Training Set')
ax.bar(['Test Diabetes'], [test_diabetes_counts], color='lightcoral', label='Testing Set')
ax.set_xlabel('Diabetes_binary')
ax.set_ylabel('Number of Patients')
ax.set_title('Distribution of Diabetes_binary (1) in Training and Testing Sets')
ax.legend()
plt.show()

In [78]:
from sklearn.preprocessing import LabelEncoder
# Заміна 99 та 97 на np.nan
diabet_dataset.replace({99: np.nan, 97: np.nan}, inplace=True)

# Конвертуємо всі категоріальні стовпці в числові
label_encoders = {}
for column in diabet_dataset.select_dtypes(include=['category', 'object']).columns:
    le = LabelEncoder()
    diabet_dataset[column] = le.fit_transform(diabet_dataset[column].astype(str))
    label_encoders[column] = le

# Заповнення відсутніх значень медіаною
diabet_dataset_median = diabet_dataset.fillna(diabet_dataset.median())

# Заповнення відсутніх значень модою
diabet_dataset_mode = diabet_dataset.apply(lambda x: x.fillna(x.mode().iloc[0]))

# Заповнення відсутніх значень середнім значенням
diabet_dataset_mean = diabet_dataset.fillna(diabet_dataset.mean())

# Видалення рядків з відсутніми значеннями
diabet_dataset_dropped = diabet_dataset.dropna()

# Заповнення відсутніх значень константою 2
diabet_dataset_constant = diabet_dataset.fillna(2)

diabet_dataset_knn = diabet_dataset.copy()

In [79]:
# Поділ даних на ознаки (X) і цільовий стовпець (y) для різних варіантів заповнення
X_median = diabet_dataset_median.drop('Diabetes_binary', axis=1)
y_median = diabet_dataset_median['Diabetes_binary']

X_mean = diabet_dataset_mean.drop('Diabetes_binary', axis=1)
y_mean = diabet_dataset_mean['Diabetes_binary']

X_mode = diabet_dataset_mode.drop('Diabetes_binary', axis=1)
y_mode = diabet_dataset_mode['Diabetes_binary']

X_deleted_nan = diabet_dataset_dropped.drop('Diabetes_binary', axis=1)
y_deleted_nan = diabet_dataset_dropped['Diabetes_binary']

X_constant = diabet_dataset_constant.drop('Diabetes_binary', axis=1)
y_constant = diabet_dataset_constant['Diabetes_binary']

# Поділ даних на навчальні і тестові набори
X_train_median, X_test_median, y_train_median, y_test_median = train_test_split(X_median, y_median, test_size=0.3, random_state=42)
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X_mean, y_mean, test_size=0.3, random_state=42)
X_train_mode, X_test_mode, y_train_mode, y_test_mode = train_test_split(X_mode, y_mode, test_size=0.3, random_state=42)
X_train_deleted_nan, X_test_deleted_nan, y_train_deleted_nan, y_test_deleted_nan = train_test_split(X_deleted_nan, y_deleted_nan, test_size=0.3, random_state=42)
X_train_constant, X_test_constant, y_train_constant, y_test_constant = train_test_split(X_constant, y_constant, test_size=0.3, random_state=42)

# Якщо ви маєте ще один набір даних для KNN, наприклад, knn_dataset, то змініть також його назву
# knn_dataset = ...
X_knn = diabet_dataset_knn.drop('Diabetes_binary', axis=1)
y_knn = diabet_dataset_knn['Diabetes_binary']

X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y_knn, test_size=0.3, random_state=42)

In [80]:
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error


In [81]:
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)),
    ('KNeighbors Classifier', KNeighborsClassifier()),
    ('Decision Tree Classifier', DecisionTreeClassifier(random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
    ('Naye base Classifier', GaussianNB())
]

datasets = [
    ('Median Imputation', X_train_median, X_test_median, y_train_median, y_test_median),
    ('Mean Imputation', X_train_mean, X_test_mean, y_train_mean, y_test_mean),
    ('Mode Imputation', X_train_mode, X_test_mode, y_train_mode, y_test_mode),
    ('Deleted NaN Imputation', X_train_deleted_nan, X_test_deleted_nan, y_train_deleted_nan, y_test_deleted_nan),
    ('Constant Imputation', X_train_constant, X_test_constant, y_train_constant, y_test_constant)
]

In [94]:
knn_classifier = KNeighborsClassifier()
logistic_regression = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)
decision_tree = DecisionTreeClassifier(random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)
naive_bayes = GaussianNB()

# List of models and their corresponding estimator objects
models = [
    ('Naive Bayes Classifier', naive_bayes)
]

# Code for imputation and splitting data goes here

# Define a function to plot accuracy for each model and imputation m
def plot_accuracy_for_ml_method(model_name, model, X_train, X_test, y_train, y_test, imputation_datasets):
    dataset_names = []
    accuracy_values = []
    print(f"Model: {model_name}")
    for dataset_name, X_train_imp, X_test_imp, y_train_imp, y_test_imp in imputation_datasets:
        pipeline = Pipeline([('imputation', 'passthrough'), ('model', model)])
        
        pipeline.fit(X_train_imp, y_train_imp)
        y_pred = pipeline.predict(X_test_imp)
        accuracy = accuracy_score(y_test_imp, y_pred)

        dataset_names.append(dataset_name)
        accuracy_values.append(accuracy)
        print("Dataset Used: ", dataset_name)
        print("Test Accuracy: ", accuracy)

    plt.figure(figsize=(8, 4))
    plt.bar(dataset_names, accuracy_values, color='lightgreen')
    plt.xlabel('Imputation Method')
    plt.ylabel('Accuracy')
    plt.title(f'Accuracy for {model_name} Model with Different Imputation Methods')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()

# Call the function for each model
for model_name, model in models:
    plot_accuracy_for_ml_method(model_name, model, X_train, X_test, y_train, y_test, datasets)

best_model = None
best_accuracy = 0.0

for dataset_name, X_train, X_test, y_train, y_test in datasets:
    print(f"Dataset: {dataset_name}")
    print("---------------------")
    for model_name, model in models:
        pipeline = Pipeline([('model', model)])

        scores = cross_val_score(pipeline, X_train, y_train, cv=5)

        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)

        print(f"Model: {model_name}")
        print("Dataset Used: ", dataset_name)
        print("Test Accuracy: ", accuracy)
        print()
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = (dataset_name, model_name, pipeline)

print("Best Model: ", best_model)

Model: Naive Bayes Classifier
Dataset Used:  Median Imputation
Test Accuracy:  0.7494813278008299
Dataset Used:  Mean Imputation
Test Accuracy:  0.7494813278008299
Dataset Used:  Mode Imputation
Test Accuracy:  0.7494813278008299
Dataset Used:  Deleted NaN Imputation
Test Accuracy:  0.7190758107612512
Dataset Used:  Constant Imputation
Test Accuracy:  0.7494813278008299
Dataset: Median Imputation
---------------------
Model: Naive Bayes Classifier
Dataset Used:  Median Imputation
Test Accuracy:  0.7494813278008299

Dataset: Mean Imputation
---------------------
Model: Naive Bayes Classifier
Dataset Used:  Mean Imputation
Test Accuracy:  0.7494813278008299

Dataset: Mode Imputation
---------------------
Model: Naive Bayes Classifier
Dataset Used:  Mode Imputation
Test Accuracy:  0.7494813278008299

Dataset: Deleted NaN Imputation
---------------------
Model: Naive Bayes Classifier
Dataset Used:  Deleted NaN Imputation
Test Accuracy:  0.7190758107612512

Dataset: Constant Imputation
----